훈련, 테스트 가져오기


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os, shutil

original_dataset_dir = '/content/drive/MyDrive/2022works/연구과제/DATA_SPECTRO'
base_dir='/content/dataset_small'
os.mkdir(base_dir)

In [3]:
train_dir=os.path.join(base_dir,'train')  
os.mkdir(train_dir) 
test_dir= os.path.join(base_dir,'test')
os.mkdir(test_dir)
T1= os.path.join(train_dir,'DTV705')
os.mkdir(T1)
T2= os.path.join(train_dir,'LTE885')
os.mkdir(T2)
t1= os.path.join(test_dir,'DTV705')
os.mkdir(t1)
t2= os.path.join(test_dir,'LTE885')
os.mkdir(t2)


In [4]:

path = '/content/drive/MyDrive/2022works/연구과제/DATA_SPECTRO/test/DTV705' # 폴더 경로
os.chdir(path) # 해당 폴더로 이동
files = os.listdir(path)
for file1 in files:
  src = os.path.join(path,file1)
  dst = os.path.join( '/content/dataset_small/test/DTV705'.format(file1),file1)
  shutil.copyfile(src,dst)

path = '/content/drive/MyDrive/2022works/연구과제/DATA_SPECTRO/test/LTE885' # 폴더 경로
os.chdir(path) # 해당 폴더로 이동
files = os.listdir(path)
for file1 in files:
  src = os.path.join(path,file1)
  dst = os.path.join( '/content/dataset_small/test/LTE885'.format(file1),file1)
  shutil.copyfile(src,dst)

path = '/content/drive/MyDrive/2022works/연구과제/DATA_SPECTRO/train/DTV705' # 폴더 경로
os.chdir(path) # 해당 폴더로 이동
files = os.listdir(path)
for file1 in files:
  src = os.path.join(path,file1)
  dst = os.path.join( '/content/dataset_small/train/DTV705'.format(file1),file1)
  shutil.copyfile(src,dst)

path = '/content/drive/MyDrive/2022works/연구과제/DATA_SPECTRO/train/LTE885' # 폴더 경로
os.chdir(path) # 해당 폴더로 이동
files = os.listdir(path)
for file1 in files:
  src = os.path.join(path,file1)
  dst = os.path.join( '/content/dataset_small/train/LTE885'.format(file1),file1)
  shutil.copyfile(src,dst)

  


# 모델링

In [5]:
from keras import layers
from keras import models

model = models. Sequential()
model.add(layers.Conv2D(32,(3,3), activation='relu',input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

# 모델의 훈련 설정하기

In [7]:
from keras import optimizers
model.compile(loss= 'binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


## 데이터 전처리

In [8]:
print(train_dir)
print(test_dir)

/content/dataset_small/train
/content/dataset_small/test


In [9]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary')


Found 1499 images belonging to 2 classes.


In [10]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary'
)

Found 500 images belonging to 2 classes.


# 학습

In [12]:

history= model.fit(
    train_generator,
    epochs=5,
    validation_data=test_generator
)

Epoch 1/5
75/75 [==============================] - 15s 70ms/step - loss: 0.2144 - acc: 0.9146 - val_loss: 0.0106 - val_acc: 1.0000
Epoch 2/5
75/75 [==============================] - 5s 65ms/step - loss: 0.0192 - acc: 0.9927 - val_loss: 0.0015 - val_acc: 1.0000
Epoch 3/5
75/75 [==============================] - 6s 82ms/step - loss: 0.0508 - acc: 0.9947 - val_loss: 3.2122e-04 - val_acc: 1.0000
Epoch 4/5
75/75 [==============================] - 5s 64ms/step - loss: 1.4090e-04 - acc: 1.0000 - val_loss: 1.8702e-04 - val_acc: 1.0000
Epoch 5/5
75/75 [==============================] - 5s 64ms/step - loss: 0.0058 - acc: 0.9967 - val_loss: 2.0812e-04 - val_acc: 1.0000


In [13]:
model.save('model1.h5')

In [14]:
test_loss, test_acc = model.evaluate(test_generator, steps=10)


10/10 [==============================] - 0s 34ms/step - loss: 2.2012e-04 - acc: 1.0000
